In [46]:
#notebook for importing and preparing data for further use
#import necessary packages
import pandas as pd

In [47]:
#import Data

#import PL data
PL_19_20 = pd.read_csv('Data/2019-20/Fixtures/E_19_20.csv')
PL_20_21 = pd.read_csv('Data/2020-21/Fixtures/E_20_21.csv')
PL_21_22 = pd.read_csv('Data/2021-22/Fixtures/E_21_22.csv')

#import xG for PL
#PL_19_20_xG = pd.read_csv('Data/2019-20/Fixtures/E_19_20_xG.csv')
#PL_20_21_xG = pd.read_csv('Data/2020-21/Fixtures/E_20_21_xG.csv')
#PL_21_22_xG = pd.read_csv('Data/2021-22/Fixtures/E_21_22_xG.csv')


In [48]:
#Merge data
PL_total = pd.concat([PL_19_20, PL_20_21, PL_21_22])
PL_total_xG = pd.concat([PL_19_20_xG, PL_20_21_xG, PL_21_22_xG])

#remove unnecessary columns
not_relevant_columns = ['BbOU','BbMx>2.5','BbAv>2.5','BbMx<2.5','B365>2.5','B365<2.5','P>2.5','P<2.5','Max>2.5','Max<2.5','Avg>2.5','Avg<2.5','BbAH','BbAHh','AHh','BbMxAHH','BbAvAHH','BbMxAHA','BbAvAHA','B365AHH','B365AHA','PAHH','PAHA','MaxAHH','MaxAHA','AvgAHH','AvgAHA','Time','MaxH','MaxD','MaxA','AvgH','AvgD','AvgA','B365CH','B365CD','B365CA','BWCH','BWCD','BWCA','IWCH','IWCD','IWCA','WHCH','WHCD','WHCA','VCCH','VCCD','VCCA','MaxCH','MaxCD','MaxCA','AvgCH','AvgCA','AvgCD','B365C>2.5','B365C<2.5','PC>2.5','PC<2.5','AHCh','B365CAHH','B365CAHA','PCAHH','PCAHA','MaxCAHH','MaxCAHA','AvgCAHH','AvgCAHA','BbAv<2.5','MaxC>2.5','MaxC<2.5','AvgC>2.5','AvgC<2.5']

#align team names
PL_19_20.replace(['Leeds', 'Leicester', 'Man City', 'Man United', 'Newcastle', 'Norwich','Sheffield United'], ['Leeds United', 'Leicester City','Manchester City', 'Manchester Utd', 'Newcastle Utd','Norwich City' ,'Sheffield Utd'], inplace=True)
PL_20_21.replace(['Leeds', 'Leicester', 'Man City', 'Man United', 'Newcastle', 'Norwich','Sheffield United'], ['Leeds United', 'Leicester City','Manchester City', 'Manchester Utd', 'Newcastle Utd','Norwich City' ,'Sheffield Utd'], inplace=True)
PL_21_22.replace(['Leeds', 'Leicester', 'Man City', 'Man United', 'Newcastle', 'Norwich','Sheffield United'], ['Leeds United', 'Leicester City','Manchester City', 'Manchester Utd', 'Newcastle Utd','Norwich City' ,'Sheffield Utd'], inplace=True)


#reset index
#PL_total.drop( axis= 1, labels=not_relevant_columns, inplace=True)
#PL_total.reset_index(inplace=False, drop=True)
#PL_total['Date'] = pd.to_datetime(PL_total['Date'], dayfirst=True)
#update team names for merging
#PL_total.replace(['Man City', 'Man United', 'Cardiff', 'Leeds', 'Leicester','Newcastle', 'Norwich','Sheffield United'], ['Manchester City', 'Manchester Utd', 'Cardiff City', 'Leeds United', 'Leicester City','Newcastle Utd','Norwich City','Sheffield Utd'], inplace=True)

#PL_20_21.replace(['Brighton', 'Leeds', 'Leicester', 'Man City', 'Man United', 'Newcastle', 'Sheffield Utd','Spurs','West Brom','West Ham', 'Wolves'], ['Brighton & Hove Albion', 'Leeds United', 'Leicester City','Manchester City', 'Manchester Utd', 'Newcastle United', 'Sheffield United','Tottenham Hotspur', 'West Bromwich Albion', 'West Ham United','Wolverhampton Wanderers'], inplace=True)



In [49]:
#Load Fifa Ratings
teams_fifa_20 = pd.read_excel('Data/2019-20/Fixtures/FIFA_2019_Team_Ratings.xlsx')
teams_fifa_21 = pd.read_excel('Data/2020-21/Fixtures/FIFA_2020_Team_Ratings.xlsx')
teams_fifa_22 = pd.read_excel('Data/2021-22/Fixtures/FIFA_2021_Team_Ratings.xlsx')

#prepare data for merge
teams_fifa_20['HomeTeam'] = teams_fifa_20['Squad']
teams_fifa_20['AwayTeam'] = teams_fifa_20['Squad']
teams_fifa_21['HomeTeam'] = teams_fifa_21['Squad']
teams_fifa_21['AwayTeam'] = teams_fifa_21['Squad']
teams_fifa_22['HomeTeam'] = teams_fifa_22['Squad']
teams_fifa_22['AwayTeam'] = teams_fifa_22['Squad']


teams_fifa_20['Rating_Home'] = teams_fifa_20['Ovr']
teams_fifa_20['Rating_Away'] = teams_fifa_20['Ovr']
teams_fifa_21['Rating_Home'] = teams_fifa_21['Ovr']
teams_fifa_21['Rating_Away'] = teams_fifa_21['Ovr']
teams_fifa_22['Rating_Home'] = teams_fifa_22['Ovr']
teams_fifa_22['Rating_Away'] = teams_fifa_22['Ovr']




In [50]:
#transform FIFA Ratings for merge
fifa_rating_home_19_20 = pd.DataFrame(teams_fifa_20, columns=['HomeTeam', 'Rating_Home'])
fifa_rating_away_19_20 = pd.DataFrame(teams_fifa_20, columns=['AwayTeam', 'Rating_Away'])
fifa_rating_home_20_21 = pd.DataFrame(teams_fifa_21, columns=['HomeTeam', 'Rating_Home'])
fifa_rating_away_20_21 = pd.DataFrame(teams_fifa_21, columns=['AwayTeam', 'Rating_Away'])
fifa_rating_home_21_22 = pd.DataFrame(teams_fifa_22, columns=['HomeTeam', 'Rating_Home'])
fifa_rating_away_21_22 = pd.DataFrame(teams_fifa_22, columns=['AwayTeam', 'Rating_Away'])

In [51]:
#merge dataframes
PL_19_20['Date'] = pd.to_datetime(PL_19_20['Date'], dayfirst=True)
PL_20_21['Date'] = pd.to_datetime(PL_20_21['Date'], dayfirst=True)
PL_21_22['Date'] = pd.to_datetime(PL_21_22['Date'], dayfirst=True)

PL_20_team_ratings = pd.merge(PL_19_20, fifa_rating_home_19_20, on='HomeTeam', how='left')
PL_20_team_ratings = pd.merge(PL_20_team_ratings, fifa_rating_away_19_20, on='AwayTeam', how='left')
PL_21_team_ratings = pd.merge(PL_20_21, fifa_rating_home_20_21, on='HomeTeam', how='left')
PL_21_team_ratings = pd.merge(PL_21_team_ratings, fifa_rating_away_20_21, on='AwayTeam', how='left')
PL_22_team_ratings = pd.merge(PL_21_22, fifa_rating_home_21_22, on='HomeTeam', how='left')
PL_22_team_ratings = pd.merge(PL_22_team_ratings, fifa_rating_away_21_22, on='AwayTeam', how='left')

PL_20_team_ratings['Rating difference'] = PL_20_team_ratings['Rating_Home']-PL_20_team_ratings['Rating_Away']
PL_21_team_ratings['Rating difference'] = PL_21_team_ratings['Rating_Home']-PL_21_team_ratings['Rating_Away']
PL_22_team_ratings['Rating difference'] = PL_22_team_ratings['Rating_Home']-PL_22_team_ratings['Rating_Away']



In [52]:
#export Data to csv
PL_20_team_ratings.to_csv('Data/2019-20/Fixtures/PL19_20_fixtures.csv',index=False)
PL_21_team_ratings.to_csv('Data/2020-21/Fixtures/PL20_21_fixtures.csv',index=False)
PL_22_team_ratings.to_csv('Data/2021-22/Fixtures/PL21_22_fixtures.csv',index=False)